# 📘 Notebook 0 — First Connection & Movement

> 🎯 **Goal**: Connect to Reachy Mini and execute your first motion commands.

---

## 0. What You Will Learn

By the end of this notebook, you will be able to:

* Understand Reachy Mini's architecture (daemon + SDK)
* Connect to your robot (real hardware or simulation)
* Execute basic motion commands
* Control the head and antennas
* Practice safe robot operation

**Duration:** 15-20 minutes

---

## 1. Architecture Overview

Reachy Mini uses a **client-server architecture**:

```
┌─────────────────┐         ┌──────────────────┐
│  Your Python    │  HTTP/  │   Reachy Daemon  │
│  Script/Notebook│ ◄─────► │   (Server)       │
│  (Client)       │  Zenoh  │                  │
└─────────────────┘         └────────┬─────────┘
                                     │
                                     ▼
                            ┌─────────────────┐
                            │  Robot Hardware │
                            │  or Simulation  │
                            └─────────────────┘
```

**Key Concepts:**

* **Daemon**: A background service that directly controls the robot's motors, sensors, camera, and audio
* **Python SDK**: The `reachy_mini` package you'll use to send commands
* **Communication**: The SDK talks to the daemon via HTTP (REST API) and Zenoh (real-time data)

**Why this architecture?**
- Multiple clients can connect simultaneously (web app, scripts, notebooks)
- The daemon handles low-level hardware safely
- You can control the robot remotely over the network. For example, you can run your AI code on a powerful server while the Daemon runs on a Raspberry Pi connected to the robot.

---

## 2. Verify Connection [TODO: rewrite this section]

Before running code, let's verify the daemon is running.

On the lite mode 

**Option A: If you're running on the robot itself (localhost):**
- Open a web browser
- Go to: `http://localhost:8000`
- You should see the Reachy Mini web interface

**Option B: If you're connecting remotely:**
- Replace `localhost` with your robot's IP address
- Example: `http://192.168.1.100:8000`

**If you don't see the interface:**
- Make sure the daemon is running, either with the desktop app 
- Check your network connection
- Refer to the [Connection and Dashboard section](https://huggingface.co/docs/reachy_mini/troubleshooting#-connection--dashboard) of the documentation for troubleshooting

---

## 3. First Connection

Let's connect to the robot from Python!

In [1]:
# Import the ReachyMini class
from reachy_mini import ReachyMini

# Connect to the robot
# Note: We're using media_backend="no_media" to skip camera/audio for now
# We'll explore media in the next notebook!
with ReachyMini(media_backend="no_media", connection_mode="auto") as mini:
    print("Successfully connected to Reachy Mini!")
    print(f"Robot name: {mini.robot_name}")

Successfully connected to Reachy Mini!
Robot name: reachy_mini


**Important Notes:**

* **`with` statement**: This is Python's context manager. It ensures the connection is properly closed when done.
* **`media_backend="no_media"`**: Disables camera and audio. We'll enable them in Notebook 1.
* **Connection modes**: By default, it tries `localhost` first, then searches the network. You can specify:
  - `connection_mode="localhost_only"` - Only connect to localhost
  - `connection_mode="network"` - Search the network for robots
  - `connection_mode="auto"` - Try localhost first, then network (default)

---

## 4. First Movement - Neutral Position

Let's move the robot to a neutral pose. This is a safe starting position.

In [2]:
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    print("Moving to neutral position...")
    
    # Move head to neutral (looking forward) and antennas to neutral (straight up)
    mini.goto_target(
        head=create_head_pose(),  # Neutral head pose (all zeros)
        antennas=[0.0, 0.0],      # Neutral antenna positions (radians)
        duration=2.0               # Take 2 seconds to complete the movement
    )
    
    print("Movement complete!")

Moving to neutral position...
Movement complete!


**What Just Happened?**

* **`goto_target()`**: Smoothly moves the robot from its current position to the target position
* **`create_head_pose()`**: Creates a neutral head pose (roll=0, pitch=0, yaw=0)
* **`antennas=[0.0, 0.0]`**: Left and right antenna positions in radians
  - `0.0` means straight up (neutral)
  - Positive values tilt outward
  - Negative values tilt inward
* **`duration=2.0`**: The movement takes 2 seconds
  - Longer duration = slower, smoother movement
  - Shorter duration = faster movement

---

## 5. Moving the Head

The head has 3 degrees of freedom (roll, pitch, yaw). Let's explore each one.

In [4]:
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # Start from neutral
    mini.goto_target(head=create_head_pose(), antennas=[0.0, 0.0], duration=1.0)
    
    # Pitch: Nod "yes" (up and down)
    print("Nodding head down...")
    mini.goto_target(
        head=create_head_pose(pitch=15, degrees=True),
        duration=1.0
    )
    
    print("Nodding head up...")
    mini.goto_target(
        head=create_head_pose(pitch=-15, degrees=True),
        duration=1.0
    )
    
    # Return to neutral
    mini.goto_target(head=create_head_pose(), duration=1.0)
    
    # Yaw: Shake "no" (left and right)
    print("Shaking head left...")
    mini.goto_target(
        head=create_head_pose(yaw=30, degrees=True),
        duration=1.0
    )
    
    print("Shaking head right...")
    mini.goto_target(
        head=create_head_pose(yaw=-30, degrees=True),
        duration=1.0
    )
    
    # Return to neutral
    mini.goto_target(head=create_head_pose(), duration=1.0)
    
    # Roll: Tilt head (like a confused look)
    print("Tilting head...")
    mini.goto_target(
        head=create_head_pose(roll=20, degrees=True),
        duration=1.0
    )
    
    # Return to neutral
    mini.goto_target(head=create_head_pose(), duration=1.0)
    
    print("Done!")

Nodding head down...
Nodding head up...
Shaking head left...
Shaking head right...
Tilting head...
Done!


[PARLER DES SAFETY LIMITS: https://github.com/pollen-robotics/reachy_mini/blob/develop/docs/SDK/core-concept.md#safety-limits-%EF%B8%8F]

**Understanding Head Rotations:**

* **Pitch**: Rotation around the X-axis (nodding up/down)
  - Positive = looking down
  - Negative = looking up

* **Yaw**: Rotation around the Z-axis (shaking left/right)
  - Positive = looking left
  - Negative = looking right

* **Roll**: Rotation around the Y-axis (tilting)
  - Positive = tilt right
  - Negative = tilt left

**Important:** Always use `degrees=True` when working with degrees! By default, the API expects radians.

---

## 6. Moving the Antennas

The antennas are great for expressing emotions and attention!

In [ ]:
import numpy as np
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # Start neutral
    mini.goto_target(head=create_head_pose(), antennas=[0.0, 0.0], duration=1.0)
    
    # Both antennas outward (excited/alert)
    print("Alert!")
    mini.goto_target(
        antennas=[-0.5, 0.5],  # Right outward (+), Left outward (-)
        duration=1.0
    )
    
    # Both antennas inward (sad/tired)
    print("Sad...")
    mini.goto_target(
        antennas=[0.5, -0.5],  # Right inward (-), Left inward (+)
        duration=1.0
    )
    
    # Return to neutral
    mini.goto_target(antennas=[0.0, 0.0], duration=1.0)
    
    # Alternating (thinking/confused)
    print("Confused?")
    mini.goto_target(antennas=[0.5, 0.5], duration=0.5)   # Right out, left in
    mini.goto_target(antennas=[-0.5, -0.5], duration=0.5) # Right in, left out
    mini.goto_target(antennas=[0.5, 0.5], duration=0.5)
    mini.goto_target(antennas=[-0.5, -0.5], duration=0.5)
    
    # Return to neutral
    mini.goto_target(antennas=[0.0, 0.0], duration=1.0)
    
    print("Done!")

Alert!
Sad...
Confused?
Done!


**Antenna Control Tips:**

* Antennas are specified as `[right, left]` in radians -> specify that for the right one : negative values tilt outward and positive values tilt inward, the left one is the opposite
* You can convert degrees to radians: `np.deg2rad(45)` or `np.radians(45)`
* Antennas are great for expressing personality!

---

## 7. Combining Head and Antenna Movements

Let's create a more complex expression by combining both!

In [7]:
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose
import numpy as np

with ReachyMini(media_backend="no_media") as mini:
    # Curious expression: tilt head + asymmetric antennas
    print("Curious...")
    mini.goto_target(
        head=create_head_pose(roll=15, pitch=-5, degrees=True),
        antennas=[0.4, 0.0],
        duration=1.5
    )
    
    # Surprised: head back + antennas wide
    print("Surprised!")
    mini.goto_target(
        head=create_head_pose(pitch=-10, degrees=True),
        antennas=np.deg2rad([30, -30]),
        duration=0.5
    )
    
    # Thinking: head tilt + one antenna up
    print("Thinking...")
    mini.goto_target(
        head=create_head_pose(roll=-10, pitch=5, degrees=True),
        antennas=[0.6, -0.2],
        duration=1.5
    )
    
    # Return to neutral
    mini.goto_target(
        head=create_head_pose(),
        antennas=[0.0, 0.0],
        duration=1.0
    )
    
    print("Done!")

Curious...
Surprised!
Thinking...
Done!


---

## 8. Safety & Best Practices

### Emergency Stop

If anything goes wrong:
* **In Jupyter**: Click the "Stop" button (■) in the toolbar, or press `Ctrl+C` in terminal
* **In Python script**: Press `Ctrl+C`
* The `with` statement ensures the robot stops safely when the context exits

### Best Practices

1. **Always use the `with` statement**
   ```python
   with ReachyMini() as mini:
       # Your code here
   # Connection automatically closed here
   ```

2. **Start with longer durations** (1-2 seconds) when experimenting
   - Slower movements are safer
   - You can speed up once you're comfortable

3. **Test small movements first**
   - Start with small angles (5-10 degrees)
   - Gradually increase range as you learn the limits

4. **Keep an eye on the robot**
   - Watch for unexpected behavior
   - Make sure the workspace is clear

5. **Use `media_backend="no_media"` when you don't need camera/audio**
   - Saves resources
   - Faster connection

### Automatic Body Yaw Mode

When you enable automatic body yaw, the robot will automatically adjust the base rotation to help the head reach targets that would otherwise be out of range.

**To enable automatic mode:**
```python
# Enable automatic body yaw when creating the robot instance
with ReachyMini(media_backend="no_media", automatic_body_yaw=True) as mini:
    # Now the robot will automatically adjust body_yaw to reach head targets
    mini.goto_target(head=create_head_pose(yaw=90, degrees=True), duration=2.0)
```

**When to use automatic mode:**
- When you want the robot to maximize its reach
- When controlling the head with look_at functions
- For interactive applications where the base should follow the head

**When to use manual control:**
- When you need precise control over both base and head
- When choreographing specific movements
- When you want independent base and head motion

**Important Notes:**

* **Base rotation limits**: The base has mechanical limits (typically ±160 degrees)
* **Smooth movements**: Use longer durations (2-3 seconds) for base rotations to avoid sudden movements
* **Stability**: The robot is most stable when the base is near the center position (0 degrees)
* **Combining motions**: You can move the head, antennas, and base all at once with a single `goto_target()` call

In [ ]:
import numpy as np
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # Scanning behavior: Turn base left while looking right
    print("Scanning left...")
    mini.goto_target(
        head=create_head_pose(yaw=-30, degrees=True),  # Head looks right
        body_yaw=np.deg2rad(45),  # Base turns left
        duration=2.0
    )
    
    # Turn base right while looking left
    print("Scanning right...")
    mini.goto_target(
        head=create_head_pose(yaw=30, degrees=True),  # Head looks left
        body_yaw=np.deg2rad(-45),  # Base turns right
        duration=2.0
    )
    
    # Return everything to neutral
    print("Returning to neutral...")
    mini.goto_target(
        head=create_head_pose(),
        body_yaw=0.0,
        duration=2.0
    )
    
    print("Scanning complete!")

### Combining Base Rotation with Head Movement

You can control the base and head simultaneously to create more complex behaviors.

In [ ]:
import numpy as np
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # Start from neutral position
    mini.goto_target(
        head=create_head_pose(),
        antennas=[0.0, 0.0],
        body_yaw=0.0,  # Base pointing forward
        duration=1.5
    )
    
    # Turn the base to the left (positive yaw)
    print("Turning base left...")
    mini.goto_target(
        body_yaw=np.deg2rad(45),  # 45 degrees to the left
        duration=2.0
    )
    
    # Turn the base to the right (negative yaw)
    print("Turning base right...")
    mini.goto_target(
        body_yaw=np.deg2rad(-45),  # 45 degrees to the right
        duration=2.0
    )
    
    # Return to center
    print("Returning to center...")
    mini.goto_target(
        body_yaw=0.0,
        duration=2.0
    )
    
    print("Done!")

---

## 7. Controlling the Base (Body Yaw)

Reachy Mini has a rotating base that allows it to turn left and right. This is controlled using the **body_yaw** parameter, which rotates the entire robot around its vertical axis.

### Understanding Body Yaw

* **Body yaw** rotates the entire base of the robot
* Measured in **radians** (or degrees with conversion)
* **Positive values** = turn left (counterclockwise when viewed from above)
* **Negative values** = turn right (clockwise when viewed from above)
* The base rotation affects the head's reachable workspace

### Automatic vs Manual Body Yaw

You can control body yaw in two ways:

1. **Manual control**: Explicitly set `body_yaw` in your commands
2. **Automatic mode**: The robot automatically adjusts the base to help the head reach targets (enabled with `automatic_body_yaw=True` when creating the ReachyMini instance)

---

## 9. Exercises (Try It Yourself!)

### Exercise 1: Create a Greeting Gesture

Create a sequence that:
1. Looks up slightly
2. Tilts head to one side
3. Wiggles antennas
4. Returns to neutral

**Hint:** Use multiple `goto_target()` calls in sequence.

<details>
<summary><b>💡 Click to reveal answer</b></summary>

```python
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # 1. Look up slightly
    mini.goto_target(
        head=create_head_pose(pitch=-10, degrees=True),
        antennas=[0.0, 0.0],
        duration=1.0
    )
    
    # 2. Tilt head to one side
    mini.goto_target(
        head=create_head_pose(roll=15, pitch=-10, degrees=True),
        duration=1.0
    )
    
    # 3. Wiggle antennas
    mini.goto_target(antennas=[0.5, -0.5], duration=0.4)
    mini.goto_target(antennas=[-0.5, 0.5], duration=0.4)
    mini.goto_target(antennas=[0.5, -0.5], duration=0.4)
    mini.goto_target(antennas=[-0.5, 0.5], duration=0.4)
    
    # 4. Return to neutral
    mini.goto_target(
        head=create_head_pose(),
        antennas=[0.0, 0.0],
        duration=1.5
    )
    
    print("Greeting complete!")
```

</details>

In [ ]:
# Your code here
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # TODO: Create your greeting gesture
    pass

### Exercise 2: Experiment with Duration

Move the head from neutral to `pitch=20°` three times with different durations:
- 0.5 seconds (fast)
- 2.0 seconds (normal)
- 5.0 seconds (slow)

Observe how the movement speed changes.

<details>
<summary><b>💡 Click to reveal answer</b></summary>

```python
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # Start from neutral
    mini.goto_target(head=create_head_pose(), duration=1.0)
    
    # Fast movement (0.5s)
    print("Fast movement (0.5s)...")
    mini.goto_target(
        head=create_head_pose(pitch=20, degrees=True),
        duration=0.5
    )
    mini.goto_target(head=create_head_pose(), duration=0.5)
    
    # Normal movement (2.0s)
    print("Normal movement (2.0s)...")
    mini.goto_target(
        head=create_head_pose(pitch=20, degrees=True),
        duration=2.0
    )
    mini.goto_target(head=create_head_pose(), duration=2.0)
    
    # Slow movement (5.0s)
    print("Slow movement (5.0s)...")
    mini.goto_target(
        head=create_head_pose(pitch=20, degrees=True),
        duration=5.0
    )
    mini.goto_target(head=create_head_pose(), duration=5.0)
    
    print("Duration experiment complete!")
```

**Observation:** Longer durations create smoother, more natural-looking movements, while shorter durations are faster but more abrupt.

</details>

In [ ]:
# Your code here
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # TODO: Test different durations
    pass

### Exercise 3: Create an "Attention" Animation

Make Reachy look like it's paying attention:
1. Start neutral
2. Look slightly to the left
3. Look slightly to the right
4. Look forward and tilt head slightly (listening pose)
5. Return to neutral

Make it smooth and natural!

<details>
<summary><b>💡 Click to reveal answer</b></summary>

```python
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # 1. Start neutral
    print("Starting attention animation...")
    mini.goto_target(
        head=create_head_pose(),
        antennas=[0.0, 0.0],
        duration=1.0
    )
    
    # 2. Look slightly to the left
    mini.goto_target(
        head=create_head_pose(yaw=25, degrees=True),
        duration=1.5
    )
    
    # 3. Look slightly to the right
    mini.goto_target(
        head=create_head_pose(yaw=-25, degrees=True),
        duration=2.0
    )
    
    # 4. Look forward and tilt head slightly (listening pose)
    mini.goto_target(
        head=create_head_pose(roll=10, pitch=5, degrees=True),
        antennas=[0.3, -0.2],  # Slightly asymmetric for curiosity
        duration=1.5
    )
    
    # 5. Return to neutral
    mini.goto_target(
        head=create_head_pose(),
        antennas=[0.0, 0.0],
        duration=2.0
    )
    
    print("Attention animation complete!")
```

**Tip:** The key to natural-looking movements is using appropriate durations and smooth transitions between poses. Experiment with different timings to find what looks best!

</details>

In [ ]:
# Your code here
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

with ReachyMini(media_backend="no_media") as mini:
    # TODO: Create attention animation
    pass

---

## 10. What's Next?

Congratulations! You've learned how to:
* ✅ Connect to Reachy Mini
* ✅ Move the head with pitch, yaw, and roll
* ✅ Control the antennas
* ✅ Create combined movements
* ✅ Follow safety best practices

In the next notebook, you'll learn:
* 📸 How to get images from the camera
* 🎤 How to record audio from the microphone
* 🔊 How to play sounds through the speaker
* 🎬 Real-time media streaming

➡️ **Next: Notebook 1 — Basic Media (Camera & Audio)** 📷🎵

---

## Appendix: Quick Reference

### Connection
```python
from reachy_mini import ReachyMini

with ReachyMini(media_backend="no_media") as mini:
    # Your code here
    pass
```

### Head Control
```python
from reachy_mini.utils import create_head_pose

# Create a head pose
pose = create_head_pose(roll=0, pitch=10, yaw=-15, degrees=True)

# Move to that pose
mini.goto_target(head=pose, duration=2.0)
```

### Antenna Control
```python
import numpy as np

# Move antennas (in radians)
mini.goto_target(antennas=[0.5, -0.5], duration=1.0)

# Or use degrees
mini.goto_target(antennas=np.deg2rad([30, -30]), duration=1.0)
```

### Combined Movement
```python
# Move head and antennas together
mini.goto_target(
    head=create_head_pose(pitch=15, degrees=True),
    antennas=[0.4, -0.4],
    duration=2.0
)
```